# Cơ chế Attention và mô hình Transformer

Một nhược điểm lớn của mạng hồi quy (recurrent networks) là tất cả các từ trong một chuỗi đều có tác động như nhau đến kết quả. Điều này dẫn đến hiệu suất không tối ưu với các mô hình LSTM encoder-decoder tiêu chuẩn cho các nhiệm vụ chuỗi sang chuỗi, chẳng hạn như Nhận diện Thực thể Được đặt tên (Named Entity Recognition) và Dịch Máy. Trong thực tế, một số từ cụ thể trong chuỗi đầu vào thường có ảnh hưởng lớn hơn đến các đầu ra tuần tự so với các từ khác.

Hãy xem xét mô hình chuỗi sang chuỗi, chẳng hạn như dịch máy. Mô hình này được triển khai bằng hai mạng hồi quy, trong đó một mạng (**encoder**) sẽ nén chuỗi đầu vào thành trạng thái ẩn, và mạng còn lại (**decoder**) sẽ mở rộng trạng thái ẩn này thành kết quả đã dịch. Vấn đề với cách tiếp cận này là trạng thái cuối cùng của mạng sẽ gặp khó khăn trong việc ghi nhớ phần đầu của câu, dẫn đến chất lượng mô hình kém đối với các câu dài.

**Cơ chế Attention** cung cấp một phương pháp để gán trọng số cho tác động ngữ cảnh của từng vector đầu vào lên từng dự đoán đầu ra của RNN. Cách triển khai là tạo các đường tắt giữa các trạng thái trung gian của RNN đầu vào và RNN đầu ra. Theo cách này, khi tạo ra ký hiệu đầu ra $y_t$, chúng ta sẽ xem xét tất cả các trạng thái ẩn đầu vào $h_i$, với các hệ số trọng số khác nhau $\alpha_{t,i}$.

![Hình ảnh mô tả mô hình encoder/decoder với lớp attention cộng tính](../../../../../translated_images/vi/encoder-decoder-attention.7a726296894fb567.webp)
*Mô hình encoder-decoder với cơ chế attention cộng tính trong [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), được trích dẫn từ [bài viết blog này](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Ma trận Attention $\{\alpha_{i,j}\}$ sẽ biểu thị mức độ mà các từ đầu vào cụ thể ảnh hưởng đến việc tạo ra một từ nhất định trong chuỗi đầu ra. Dưới đây là ví dụ về một ma trận như vậy:

![Hình ảnh minh họa một mẫu căn chỉnh được tìm thấy bởi RNNsearch-50, lấy từ Bahdanau - arviz.org](../../../../../translated_images/vi/bahdanau-fig3.09ba2d37f202a6af.webp)

*Hình ảnh được lấy từ [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Hình 3)*

Cơ chế Attention chịu trách nhiệm cho phần lớn các thành tựu hiện tại hoặc gần đây trong Xử lý Ngôn ngữ Tự nhiên. Tuy nhiên, việc thêm Attention làm tăng đáng kể số lượng tham số của mô hình, dẫn đến các vấn đề về khả năng mở rộng với RNNs. Một hạn chế chính của việc mở rộng RNNs là tính chất hồi quy của các mô hình khiến việc xử lý theo lô (batch) và song song hóa (parallelize) trở nên khó khăn. Trong RNN, mỗi phần tử của một chuỗi cần được xử lý theo thứ tự tuần tự, điều này có nghĩa là không thể dễ dàng song song hóa.

Việc áp dụng cơ chế Attention kết hợp với hạn chế này đã dẫn đến sự ra đời của các Mô hình Transformer hiện đại mà chúng ta biết và sử dụng ngày nay, từ BERT đến OpenGPT3.

## Mô hình Transformer

Thay vì chuyển tiếp ngữ cảnh của từng dự đoán trước vào bước đánh giá tiếp theo, **mô hình Transformer** sử dụng **mã hóa vị trí** (positional encodings) và **attention** để nắm bắt ngữ cảnh của đầu vào trong một cửa sổ văn bản được cung cấp. Hình ảnh dưới đây minh họa cách mã hóa vị trí kết hợp với attention có thể nắm bắt ngữ cảnh trong một cửa sổ nhất định.

![GIF động minh họa cách các đánh giá được thực hiện trong mô hình Transformer.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Vì mỗi vị trí đầu vào được ánh xạ độc lập đến mỗi vị trí đầu ra, các mô hình Transformer có thể song song hóa tốt hơn so với RNNs, điều này cho phép tạo ra các mô hình ngôn ngữ lớn hơn và biểu đạt hơn. Mỗi attention head có thể được sử dụng để học các mối quan hệ khác nhau giữa các từ, cải thiện các nhiệm vụ Xử lý Ngôn ngữ Tự nhiên.

## Xây dựng Mô hình Transformer Đơn giản

Keras không chứa lớp Transformer tích hợp sẵn, nhưng chúng ta có thể tự xây dựng. Như trước đây, chúng ta sẽ tập trung vào phân loại văn bản của tập dữ liệu AG News, nhưng cần lưu ý rằng các mô hình Transformer cho kết quả tốt nhất ở các nhiệm vụ NLP khó hơn.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Các lớp mới trong Keras cần kế thừa lớp `Layer`, và triển khai phương thức `call`. Hãy bắt đầu với lớp **Positional Embedding**. Chúng ta sẽ sử dụng [một số mã từ tài liệu chính thức của Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Chúng ta sẽ giả định rằng tất cả các chuỗi đầu vào đều được đệm đến độ dài `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Lớp này bao gồm hai lớp `Embedding`: một lớp để nhúng các token (theo cách chúng ta đã thảo luận trước đó) và một lớp để nhúng vị trí của các token. Vị trí của các token được tạo thành một dãy số tự nhiên từ 0 đến `maxlen` bằng cách sử dụng `tf.range`, sau đó được đưa qua lớp nhúng. Hai vector nhúng kết quả sau đó được cộng lại, tạo ra biểu diễn nhúng theo vị trí của đầu vào với hình dạng `maxlen`$\times$`embed_dim`.

Bây giờ, chúng ta hãy triển khai khối transformer. Nó sẽ nhận đầu ra từ lớp nhúng đã được định nghĩa trước đó:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Transformer áp dụng `MultiHeadAttention` lên đầu vào đã được mã hóa vị trí để tạo ra vector attention với kích thước `maxlen`$\times$`embed_dim`, sau đó được trộn với đầu vào và chuẩn hóa bằng cách sử dụng `LayerNormalization`.

> **Lưu ý**: `LayerNormalization` tương tự như `BatchNormalization` đã được thảo luận trong phần *Thị giác Máy tính* của lộ trình học này, nhưng nó chuẩn hóa đầu ra của lớp trước đó cho từng mẫu huấn luyện một cách độc lập, để đưa chúng về phạm vi [-1..1].

Đầu ra của lớp này sau đó được truyền qua mạng `Dense` (trong trường hợp của chúng ta - perceptron hai lớp), và kết quả được cộng vào đầu ra cuối cùng (đầu ra này lại được chuẩn hóa một lần nữa).

Bây giờ, chúng ta đã sẵn sàng để định nghĩa mô hình transformer hoàn chỉnh:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Các Mô Hình Transformer BERT

**BERT** (Bidirectional Encoder Representations from Transformers) là một mạng transformer nhiều lớp rất lớn với 12 lớp cho *BERT-base*, và 24 lớp cho *BERT-large*. Mô hình này được huấn luyện trước trên một tập dữ liệu văn bản lớn (WikiPedia + sách) bằng cách sử dụng phương pháp huấn luyện không giám sát (dự đoán các từ bị che trong câu). Trong quá trình huấn luyện trước, mô hình hấp thụ một mức độ hiểu biết ngôn ngữ đáng kể, sau đó có thể được tận dụng với các tập dữ liệu khác thông qua việc tinh chỉnh. Quá trình này được gọi là **học chuyển giao**.

![hình ảnh từ http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/vi/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Có nhiều biến thể của kiến trúc Transformer bao gồm BERT, DistilBERT, BigBird, OpenGPT3 và nhiều hơn nữa có thể được tinh chỉnh.

Hãy cùng xem cách chúng ta có thể sử dụng mô hình BERT đã được huấn luyện trước để giải quyết vấn đề phân loại chuỗi truyền thống. Chúng ta sẽ mượn ý tưởng và một số đoạn mã từ [tài liệu chính thức](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Để tải các mô hình đã được huấn luyện trước, chúng ta sẽ sử dụng **Tensorflow hub**. Đầu tiên, hãy tải vectorizer dành riêng cho BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Điều quan trọng là bạn phải sử dụng cùng bộ vector hóa như bộ mà mạng ban đầu đã được huấn luyện. Ngoài ra, bộ vector hóa BERT trả về ba thành phần:
* `input_word_ids`, là một chuỗi các số token cho câu đầu vào
* `input_mask`, cho biết phần nào của chuỗi chứa đầu vào thực tế và phần nào là padding. Nó tương tự như mặt nạ được tạo bởi lớp `Masking`
* `input_type_ids` được sử dụng cho các nhiệm vụ mô hình ngôn ngữ, và cho phép chỉ định hai câu đầu vào trong một chuỗi.

Sau đó, chúng ta có thể khởi tạo bộ trích xuất đặc trưng BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Vậy, lớp BERT trả về một số kết quả hữu ích:

* `pooled_output` là kết quả của việc tính trung bình tất cả các token trong chuỗi. Bạn có thể xem nó như một embedding ngữ nghĩa thông minh của toàn bộ mạng. Nó tương đương với đầu ra của lớp `GlobalAveragePooling1D` trong mô hình trước của chúng ta.
* `sequence_output` là đầu ra của lớp transformer cuối cùng (tương ứng với đầu ra của `TransformerBlock` trong mô hình ở trên).
* `encoder_outputs` là đầu ra của tất cả các lớp transformer. Vì chúng ta đã tải mô hình BERT 4 lớp (như bạn có thể đoán từ tên gọi, chứa `4_H`), nên nó có 4 tensor. Tensor cuối cùng giống với `sequence_output`.

Bây giờ chúng ta sẽ định nghĩa mô hình phân loại end-to-end. Chúng ta sẽ sử dụng *định nghĩa mô hình chức năng*, khi chúng ta định nghĩa đầu vào của mô hình, và sau đó cung cấp một loạt các biểu thức để tính toán đầu ra của nó. Chúng ta cũng sẽ đặt trọng số của mô hình BERT ở trạng thái không thể huấn luyện, và chỉ huấn luyện bộ phân loại cuối cùng:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Mặc dù có rất ít tham số có thể huấn luyện, quá trình vẫn khá chậm vì bộ trích xuất đặc trưng của BERT tiêu tốn nhiều tài nguyên tính toán. Có vẻ như chúng ta không thể đạt được độ chính xác hợp lý, có thể do thiếu huấn luyện hoặc thiếu tham số của mô hình.

Hãy thử mở khóa các trọng số của BERT và huấn luyện nó. Điều này yêu cầu tốc độ học rất nhỏ, cùng với chiến lược huấn luyện cẩn thận hơn với **warmup**, sử dụng trình tối ưu hóa **AdamW**. Chúng ta sẽ sử dụng gói `tf-models-official` để tạo trình tối ưu hóa:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Như bạn có thể thấy, quá trình huấn luyện diễn ra khá chậm - nhưng bạn có thể thử nghiệm và huấn luyện mô hình trong vài epoch (5-10) để xem liệu bạn có thể đạt được kết quả tốt nhất so với các phương pháp mà chúng ta đã sử dụng trước đây hay không.

## Thư viện Huggingface Transformers

Một cách khác rất phổ biến (và đơn giản hơn một chút) để sử dụng các mô hình Transformer là [gói HuggingFace](https://github.com/huggingface/), cung cấp các khối xây dựng đơn giản cho các tác vụ NLP khác nhau. Thư viện này có sẵn cho cả Tensorflow và PyTorch, một framework mạng nơ-ron rất phổ biến khác.

> **Lưu ý**: Nếu bạn không quan tâm đến việc tìm hiểu cách thư viện Transformers hoạt động - bạn có thể bỏ qua phần cuối của notebook này, vì bạn sẽ không thấy điều gì khác biệt đáng kể so với những gì chúng ta đã làm ở trên. Chúng ta sẽ lặp lại các bước tương tự để huấn luyện mô hình BERT bằng một thư viện khác và một mô hình lớn hơn đáng kể. Do đó, quá trình này bao gồm một số bước huấn luyện khá dài, vì vậy bạn có thể chỉ cần xem qua mã.

Hãy cùng xem cách chúng ta có thể giải quyết vấn đề của mình bằng [Huggingface Transformers](http://huggingface.co).


Điều đầu tiên chúng ta cần làm là chọn mô hình mà chúng ta sẽ sử dụng. Ngoài một số mô hình tích hợp sẵn, Huggingface còn có một [kho lưu trữ mô hình trực tuyến](https://huggingface.co/models), nơi bạn có thể tìm thấy rất nhiều mô hình đã được huấn luyện trước bởi cộng đồng. Tất cả các mô hình này đều có thể được tải và sử dụng chỉ bằng cách cung cấp tên mô hình. Tất cả các tệp nhị phân cần thiết cho mô hình sẽ tự động được tải xuống.

Trong một số trường hợp, bạn sẽ cần tải mô hình của riêng mình, khi đó bạn có thể chỉ định thư mục chứa tất cả các tệp liên quan, bao gồm các tham số cho tokenizer, tệp `config.json` với các tham số mô hình, trọng số nhị phân, v.v.

Từ tên mô hình, chúng ta có thể khởi tạo cả mô hình và tokenizer. Hãy bắt đầu với tokenizer:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Đối tượng `tokenizer` chứa hàm `encode` có thể được sử dụng trực tiếp để mã hóa văn bản:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Chúng ta cũng có thể sử dụng tokenizer để mã hóa một chuỗi theo cách phù hợp để truyền vào mô hình, tức là bao gồm các trường `token_ids`, `input_mask`, v.v. Chúng ta cũng có thể chỉ định rằng chúng ta muốn các tensor của Tensorflow bằng cách cung cấp đối số `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Trong trường hợp này, chúng ta sẽ sử dụng mô hình BERT đã được huấn luyện trước, gọi là `bert-base-uncased`. *Uncased* cho biết rằng mô hình không phân biệt chữ hoa chữ thường.

Khi huấn luyện mô hình, chúng ta cần cung cấp chuỗi đã được mã hóa làm đầu vào, và do đó chúng ta sẽ thiết kế quy trình xử lý dữ liệu. Vì `tokenizer.encode` là một hàm Python, chúng ta sẽ sử dụng cách tiếp cận tương tự như trong bài học trước bằng cách gọi nó thông qua `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Bây giờ chúng ta có thể tải mô hình thực tế bằng gói `BertForSequenceClassification`. Điều này đảm bảo rằng mô hình của chúng ta đã có kiến trúc cần thiết cho phân loại, bao gồm cả bộ phân loại cuối cùng. Bạn sẽ thấy thông báo cảnh báo cho biết rằng các trọng số của bộ phân loại cuối cùng chưa được khởi tạo, và mô hình sẽ cần được huấn luyện trước - điều đó hoàn toàn ổn, vì đó chính xác là những gì chúng ta sắp làm!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Như bạn có thể thấy từ `summary()`, mô hình chứa gần 110 triệu tham số! Có lẽ, nếu chúng ta muốn thực hiện nhiệm vụ phân loại đơn giản trên tập dữ liệu tương đối nhỏ, chúng ta không muốn huấn luyện lớp cơ sở BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Bây giờ chúng ta sẵn sàng bắt đầu huấn luyện!

> **Lưu ý**: Việc huấn luyện mô hình BERT toàn diện có thể mất rất nhiều thời gian! Vì vậy, chúng ta sẽ chỉ huấn luyện nó trong 32 lô đầu tiên. Đây chỉ là để minh họa cách thiết lập huấn luyện mô hình. Nếu bạn muốn thử huấn luyện toàn diện - chỉ cần loại bỏ các tham số `steps_per_epoch` và `validation_steps`, và chuẩn bị chờ đợi!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Nếu bạn tăng số lượng vòng lặp và chờ đủ lâu, đồng thời huấn luyện trong vài epoch, bạn có thể kỳ vọng rằng phân loại bằng BERT sẽ mang lại độ chính xác tốt nhất! Điều này là do BERT đã hiểu khá tốt cấu trúc của ngôn ngữ, và chúng ta chỉ cần tinh chỉnh bộ phân loại cuối cùng. Tuy nhiên, vì BERT là một mô hình lớn, toàn bộ quá trình huấn luyện mất rất nhiều thời gian và đòi hỏi sức mạnh tính toán đáng kể! (GPU, và tốt nhất là nhiều hơn một GPU).

> **Lưu ý:** Trong ví dụ của chúng ta, chúng ta đã sử dụng một trong những mô hình BERT nhỏ nhất đã được huấn luyện sẵn. Có những mô hình lớn hơn có khả năng mang lại kết quả tốt hơn.


## Kết luận

Trong bài học này, chúng ta đã tìm hiểu về các kiến trúc mô hình rất mới dựa trên **transformers**. Chúng ta đã áp dụng chúng cho nhiệm vụ phân loại văn bản, nhưng tương tự, các mô hình BERT cũng có thể được sử dụng cho việc trích xuất thực thể, trả lời câu hỏi, và các nhiệm vụ NLP khác.

Các mô hình transformer hiện đang đại diện cho trạng thái tiên tiến nhất trong NLP, và trong hầu hết các trường hợp, đây nên là giải pháp đầu tiên bạn bắt đầu thử nghiệm khi triển khai các giải pháp NLP tùy chỉnh. Tuy nhiên, việc hiểu các nguyên tắc cơ bản của mạng nơ-ron hồi quy được thảo luận trong module này là vô cùng quan trọng nếu bạn muốn xây dựng các mô hình nơ-ron tiên tiến.



---

**Tuyên bố miễn trừ trách nhiệm**:  
Tài liệu này đã được dịch bằng dịch vụ dịch thuật AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mặc dù chúng tôi cố gắng đảm bảo độ chính xác, xin lưu ý rằng các bản dịch tự động có thể chứa lỗi hoặc không chính xác. Tài liệu gốc bằng ngôn ngữ bản địa nên được coi là nguồn thông tin chính thức. Đối với các thông tin quan trọng, nên sử dụng dịch vụ dịch thuật chuyên nghiệp từ con người. Chúng tôi không chịu trách nhiệm cho bất kỳ sự hiểu lầm hoặc diễn giải sai nào phát sinh từ việc sử dụng bản dịch này.
